<a href="https://colab.research.google.com/github/LambertLeong/ICS635_ML/blob/master/Quasars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
# Build simple model for classifying quasars, galaxies, and stars. 
# Author Peter Sadowski
# Adapted from https://www.kaggle.com/lucidlenn/sloan-digital-sky-survey/home

import urllib
import numpy as np
import pandas as pd
import sklearn
import sklearn.model_selection
import sklearn.metrics
import sklearn.ensemble
import matplotlib.pyplot as plt

# Download data from Sloan Digital Sky Survey 
# https://www.sdss.org/dr14/
url = 'https://raw.githubusercontent.com/peterjsadowski/sklearn_examples/master/sdss/sdss.csv'
filename = 'sdss.csv'
urllib.request.urlretrieve(url, filename)

('sdss.csv', <http.client.HTTPMessage at 0x7f66755b5f60>)

In [65]:
# Features:
# ra = J2000 Right Ascension (r-band)
# dec = J2000 Declination (r-band)
# u = better of DeV/Exp magnitude fit
# g = better of DeV/Exp magnitude fit
# r = better of DeV/Exp magnitude fit
# i = better of DeV/Exp magnitude fit
# z = better of DeV/Exp magnitude fit
# redshift = Redshift
# plate = plate number
# mjd = MJD of observation
# fiberid = fiber ID

# The Thuan-Gunn astronomic magnitude system. u, g, r, i, z represent the response of the 5 bands of the telescope.
# Redshift is the change in electromagnetic radiation due to the object moving away from the observer.

data = pd.read_csv(filename)
data.drop(["objid","specobjid","run","rerun","camcol","field"], axis = 1, inplace = True) # Unused columns.

print(data.head(n=5))
print(data.info())

           ra       dec         u         g         r         i         z  \
0  183.531326  0.089693  19.47406  17.04240  15.94699  15.50342  15.22531   
1  183.598371  0.135285  18.66280  17.21449  16.67637  16.48922  16.39150   
2  183.680207  0.126185  19.38298  18.19169  17.47428  17.08732  16.80125   
3  183.870529  0.049911  17.76536  16.60272  16.16116  15.98233  15.90438   
4  183.883288  0.102557  17.55025  16.26342  16.43869  16.55492  16.61326   

    class  redshift  plate    mjd  fiberid  
0    STAR -0.000009   3306  54922      491  
1    STAR -0.000055    323  51615      541  
2  GALAXY  0.123111    287  52023      513  
3    STAR -0.000111   3306  54922      510  
4    STAR  0.000590   3306  54922      512  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
ra          10000 non-null float64
dec         10000 non-null float64
u           10000 non-null float64
g           10000 non-null float64
r           10000 n

In [141]:
# Preprocess data.

# Associate each class with a number.
print("Mapping: ", dict(enumerate(["GALAXY","QUASAR","STAR"])))
data["class"] = data["class"].astype("category")
data["class"] = data["class"].cat.codes
print(data["class"].value_counts().sort_index())

# Split data set.
features = data.drop("class", axis = 1)
labels = data["class"].copy()
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(features, labels, test_size=0.2, random_state=42, stratify=labels)
#X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=None)
print('Train data shape:', X_train.shape, y_train.shape)
#print('Val data shape:', X_val.shape, y_val.shape)
print('Test data shape:' , X_test.shape, y_test.shape)


Mapping:  {0: 'GALAXY', 1: 'QUASAR', 2: 'STAR'}
0    4998
1     850
2    4152
Name: class, dtype: int64
Train data shape: (8000, 11) (8000,)
Test data shape: (2000, 11) (2000,)


In [0]:
#Feature Selection
from sklearn.feature_selection import SelectKBest
new_x_train = SelectKBest(k=2).fit_transform(X_train, y_train)
new_y_train = y_train.values.tolist()
#print(new_y_train)

In [0]:
if 0: #useless
  # Plot 
  #print(X_train[0])
  cls0_0 = [new_x_train[ii][0] for ii in range(0, len(new_x_train)) if new_y_train[ii]==0]
  cls1_0 = [new_x_train[ii][0] for ii in range(0, len(new_x_train)) if new_y_train[ii]==1]
  cls2_0 = [new_x_train[ii][0] for ii in range(0, len(new_x_train)) if new_y_train[ii]==2]
  cls0_1 = [new_x_train[ii][1] for ii in range(0, len(new_x_train)) if new_y_train[ii]==0]
  cls1_1 = [new_x_train[ii][1] for ii in range(0, len(new_x_train)) if new_y_train[ii]==1]
  cls2_1 = [new_x_train[ii][1] for ii in range(0, len(new_x_train)) if new_y_train[ii]==2]
  #bumpy_slow = [features_train[ii][1] for ii in range(0, len(features_train)) if labels_train[ii]==1]

  #print(len(cls0_1))
  #### initial visualization
  #plt.xlim(0.0, 1.0)
  #plt.ylim(0.0, 1.0)
  plt.scatter(cls0_0, cls0_1 , color = "b", label="cls0")
  plt.scatter(cls1_0, cls1_1, color = "r", label="cls1")
  plt.scatter(cls2_0, cls2_1, color = "g", label="cls1")
  plt.legend()
  plt.xlabel("feat 1")
  plt.ylabel("feat 2")
  plt.show()


In [0]:
#sample_weights
s_weights = []
for i in range(0,len(new_y_train )):
  if new_y_train[i] == 0:
    s_weights.append(1)
  elif new_y_train[i] == 1:
    s_weights.append(5.88)
  elif new_y_train[i] == 2:
     s_weights.append(1.20)
#print((y_train))
#print(max(s_weights))

In [132]:
# Train and test classifier:
import sklearn.neighbors
from sklearn.model_selection import cross_val_score
classifier = sklearn.neighbors.KNeighborsClassifier(n_neighbors=9, algorithm = 'auto', radius = 1)
classifier.fit(X_train, y_train)
scores = cross_val_score(classifier, features, labels, cv=100)
accuracy = classifier.score(X_test, y_test)
print("CV accuracy: %0.4f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print(f'Test accuracy: {accuracy}')

CV accuracy: 0.7674 (+/- 0.16)
Test accuracy: 0.7865


In [163]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb = gnb.fit(X_train, y_train)#[1, 5.88, 1.20] [0:1, 1:5.88, 2:1.20]
accuracy = gnb.score(X_test, y_test,) 
scores = cross_val_score(gnb, features, labels, cv=100)
print("GNB CV Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))
print(f'GNB Test accuracy: {accuracy}')

p_gnb = gnb.partial_fit(X_train, y_train, sample_weight=s_weights)
scores = cross_val_score(p_gnb, features, labels, cv=100)
print("Partial GNB CV Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))
p_accuracy = p_gnb.score(X_test, y_test)
print(f'partial GNB Test accuracy: {p_accuracy}')


GNB CV Accuracy: 0.8563 (+/- 0.1516)
GNB Test accuracy: 0.8625
Partial GNB CV Accuracy: 0.8563 (+/- 0.1516)
partial GNB Test accuracy: 0.8535


In [171]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
LDA = LinearDiscriminantAnalysis(priors=None, shrinkage=None, solver='svd', store_covariance=False, tol=0.0001)
#LDA = LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None, solver='lsqr', store_covariance=False, tol=0.0001)
LDA.fit(X_train, y_train)
accuracy = LDA.score(X_test, y_test)
scores = cross_val_score(LDA, features, labels, cv=100)
print("LDA CV Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))
print(f'partial LDA Test accuracy: {accuracy}')

LDA CV Accuracy: 0.9073 (+/- 0.1023)
partial LDA Test accuracy: 0.892


In [173]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
qda = QuadraticDiscriminantAnalysis(priors=None, reg_param=0.0,
                              store_covariance=False,
                              store_covariances=None, tol=0.0001)
qda.fit(X_train, y_train)
scores = cross_val_score(qda, features, labels, cv=100)
accuracy = qda.score(X_test, y_test)
print("qda CV Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))
print(f'qda Test accuracy: {accuracy}')

qda CV Accuracy: 0.9812 (+/- 0.0321)
qda Test accuracy: 0.9815


In [182]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth = 3)
tree = tree.fit(X_train, y_train)
scores = cross_val_score(tree, features, labels, cv=100)
accuracy = tree.score(X_test, y_test)
print("dTree CV Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))
print(f'dTree Test accuracy: {accuracy}')

dTree CV Accuracy: 0.9873 (+/- 0.0269)
dTree Test accuracy: 0.991


In [184]:
from sklearn.ensemble import GradientBoostingClassifier
xgb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=3, random_state=0)
xgb = xgb.fit(X_train, y_train)
scores = cross_val_score(xgb, features, labels, cv=100)
accuracy = xgb.score(X_test, y_test)
print("dTree CV Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))
print(f'dTree Test accuracy: {accuracy}')

dTree CV Accuracy: 0.9428 (+/- 0.3020)
dTree Test accuracy: 0.989
